## Molecular dynamics simulations of bulk water
I followed the officail tutorial to simulate bulk water

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pylab as plt
from pyiron import Project
import ase.units as units
import pandas

In [2]:
pr = Project("tip3p_water")

In [3]:
density = 1.0e-24  # g/A^3
n_mols = 27
mol_mass_water = 18.015 # g/mol

# Determining the supercell size size
mass = mol_mass_water * n_mols / units.mol  # g
vol_h2o = mass / density # in A^3
a = vol_h2o ** (1./3.) # A

# Constructing the unitcell
n = int(round(n_mols ** (1. / 3.)))

dx = 0.7
r_O = [0, 0, 0]
r_H1 = [dx, dx, 0]
r_H2 = [-dx, dx, 0]
unit_cell = (a / n) * np.eye(3)
water = pr.create_atoms(elements=['H', 'H', 'O'],
                        positions=[r_H1, r_H2, r_O],
                        cell=unit_cell, pbc=True)
water.set_repeat([n, n, n])
water.plot3d()

NGLWidget()

In [4]:
water_potential = pandas.DataFrame({
    'Name': ['H2O_tip3p'],
    'Filename': [[]],
    'Model': ["TIP3P"],
    'Species': [['H', 'O']],
    'Config': [['# @potential_species H_O ### species in potential\n', '# W.L. Jorgensen et.al., The Journal of Chemical Physics 79, 926 (1983); https://doi.org/10.1063/1.445869\n', '#\n', '\n', 'units real\n', 'dimension 3\n', 'atom_style full\n', '\n', '# create groups ###\n', 'group O type 2\n', 'group H type 1\n', '\n', '## set charges - beside manually ###\n', 'set group O charge -0.830\n', 'set group H charge 0.415\n', '\n', '### TIP3P Potential Parameters ###\n', 'pair_style lj/cut/coul/long 10.0\n', 'pair_coeff * * 0.0 0.0 \n', 'pair_coeff 2 2 0.102 3.188 \n', 'bond_style harmonic\n', 'bond_coeff 1 450 0.9572\n', 'angle_style harmonic\n', 'angle_coeff 1 55 104.52\n', 'kspace_style pppm 1.0e-5\n', '\n']]
})

In [5]:
job_name = "water_slow"
ham = pr.create_job("Lammps", job_name)
ham.structure = water
ham.potential = water_potential

/Users/mac/.jackprogram/anaconda3/lib/python3.9/site-packages/pyiron_atomistics/lammps/base.py:185: UserWarning: WARNING: Non-'metal' units are not fully supported. Your calculation should run OK, but results may not be saved in pyiron units.
  warnings.warn(


In [6]:
ham.calc_md(temperature=300,
            n_ionic_steps=1e4,
            time_step=0.01)
ham.run()

/Users/mac/.jackprogram/anaconda3/lib/python3.9/site-packages/pyiron_atomistics/lammps/structure.py:434: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for j, ind in enumerate(np.array(neighbors.indices)[el_1_list]):
/Users/mac/.jackprogram/anaconda3/lib/python3.9/site-packages/pyiron_atomistics/lammps/structure.py:439: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bool_1 = np.array(neighbors.distances)[id_el] <= cutoff_dist


The job water_slow was saved and received the ID: 48


In [7]:
view = ham.animate_structure()
view

NGLWidget(max_frame=100)

In [8]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,48,finished,H54O27,water_slow,/water_slow,None,/Volumes/SD/Github/HB-Life/tip3p_water/,2022-03-18 21:54:25.077398,2022-03-18 21:54:35.921646,10.0,pyiron@Macbook.local#1,Lammps,0.1,None,None


I use the following way because I have multiple types of simulations, like SPE, TIP4P, so that I can use different initio configuarations in a loop, and can use these configuarations for the next time. 

In [9]:
job = pr.load('water_slow')
job.decompress()

In [10]:
job.animate_structure() # Works okey

NGLWidget(max_frame=100)

In [11]:
job['output/generic/']

{'groups': [], 'nodes': ['cells', 'energy_pot', 'energy_tot', 'forces', 'indices', 'positions', 'pressures', 'steps', 'temperature', 'unwrapped_positions', 'velocities', 'volume']}

In [13]:
 #  job['output/generic/positions'] # Work


In [16]:
# job.get_structure() # Not work
# job.get_final_structure() # Not work
job.get_structure(iteration_step=0) # Not work

TypeError: No loop matching the specified signature and casting was found for ufunc solve

In [17]:
job.number_of_structures

AttributeError: 'Lammps' object has no attribute 'number_of_structures'

On my machine, this job only has two attributes start with letter 'n', 'name' and 'next'.

In [18]:
job.name

'water_slow'

In [19]:
job.next

<bound method LammpsBase.next of {'groups': ['input', 'output'], 'nodes': ['HDF_VERSION', 'NAME', 'TYPE', 'VERSION', 'server', 'status']}>